## PCA 

In [ ]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint

tickers = [
    # Original list
    "AMD", "NVDA", "AVGO", "MU", "TSM", "QCOM", "NET",
    "SNOW", "AMZN", "MSFT", "GOOGL", "DDOG", "ESTC", "DOCN", "PLTR", "MDB", 
    "OKTA", "CRWD", "ZS", "PANW", "S", "FTNT", "CYBR", "U",
    "TTD", "ROKU", "PINS", "SNAP", "PUBM", "CFLT", "GTLB", "FROG", "PD",
    "ORCL",
    
    # Additional cybersecurity
    "CHKP", "QLYS", "RPD", "TENB", "SAIL", "VRNS", "AKAM", "CACI",
    
    # Cloud/SaaS Software
    "WDAY", "VEEV", "ADSK", "INTU", "SHOP", "ZM", "DOCU", 
    "BOX", "DBX", "TWLO", "PYPL", "ADBE", "NOW", "HUBS",
    
    # Data/Analytics
    "DOMO", 
    
    # Semiconductors
    "ON", "MCHP", "SWKS", "QRVO", "LRCX", "KLAC", 
    "AMAT", "ASML",
    
    # Cloud Infrastructure
    "FSLY", "DT",
    
    # Communication/Collaboration
    "RING",
    
    # Fintech/Digital Payments  
    "AFRM", "COIN", "HOOD", "UPST"
]

print(f"Total stocks: {len(tickers)}")



# Download price data

data = yf.download(tickers, period="2y")['Close']
portfolio_returns = data['Adj Close'].pct_change().dropna()
portfolio_return.to_csv = 


Total stocks: 72


C:\Users\Michael Gray\AppData\Local\Temp\ipykernel_1396\3608872575.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="2y")['Close']
[*********************100%***********************]  72 of 72 completed

3 Failed downloads:
['NET', 'GOOGL']: Timeout('Failed to perform, curl: (28) Connection timed out after 10011 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['FROG']: Timeout('Failed to perform, curl: (28) Connection timed out after 10007 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


KeyError: 'Adj Close'